In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from collections import Counter
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import shutil
import random

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
DATASET_ROOT = "/content/drive/MyDrive/Flower_Classification/Flowers50_Split"

TRAIN_DIR = os.path.join(DATASET_ROOT, "train")
VAL_DIR   = os.path.join(DATASET_ROOT, "val")
TEST_DIR  = os.path.join(DATASET_ROOT, "test")

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 16
SEED = 42

In [ ]:
# Load datasets (directory-level shuffle is enough)

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)

print("Classes:", num_classes)

In [ ]:
# Preprocessing

def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, label

train_ds = train_ds.map(preprocess).prefetch(1)
val_ds   = val_ds.map(preprocess).prefetch(1)
test_ds  = test_ds.map(preprocess).prefetch(1)

In [ ]:
# Load Pre-Trained Model

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False

In [ ]:
# Model Building

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(num_classes, activation="softmax")
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# Train

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=5,
            restore_best_weights=True
        )
    ]
)

In [ ]:
# Evaluate

test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)

In [ ]:
# Export for Flutter

model.export("flowers50_model")

converter = tf.lite.TFLiteConverter.from_saved_model("flowers50_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [ ]:
OUTPUT_DIR = "/content/drive/MyDrive/Flower_Classification"

with open(f"{OUTPUT_DIR}/flowers50_model.tflite", "wb") as f:
    f.write(tflite_model)

with open(f"{OUTPUT_DIR}/labels.txt", "w") as f:
    for label in class_names:
        f.write(label + "\n")

print("Model and labels saved.")